# What is Topic Modeling?

Topic modeling is a way to figure out the underlying structure of a piece of text. It tries to represent a text by a collection of few concepts (or topics) rather than a whole bunch of words. In other words, we assign tags to a document so that it can be compared with other documents or be put into categories. It can be used in recommendation systems by suggesting a similar article after one has read an article.

---

# What should I know before TM?

##### Document-term matrix, term-document matrix, count matrix
A matrix whose rows and columns correspond to documents and terms. The entries correspond to the frequency of the term in the document. Sometimes the rows correspond to terms and columns correspond to terms.

Here's what it could look like,

|...|term1|term2|term3|term4|
|---|:---:|:---:|:---:|:---:|
|d1 |1    |0    |4    |2    |
|d2 |6    |2    |1    |0    |
|d3 |4    |3    |0    |0    |


##### Term frequency (tf)
The number of times a term appears in a document. It gives all terms equal importance and so cannot be used by itself to understand the importance of a term in a document. Words that appear a lot, in many documents (high tf - low "specificity") can be words that do not impact the meaning of a document significantly. Therefore tf can be used to identify stop-words. 

tf can either be a raw-count of a term in a document or expressed as a fraction of raw-count to total number of terms in the document.

$$
\begin{equation}
\mathrm{tf}(t, d) = \frac{\mathrm{n}(t, d)}{\sum _{i \in d}{\mathrm{n}(i, d)}}
\end{equation}
$$

where 't' is a term, 'd' is a document, 'n' gives the frequency of the term in the document, and 'i' represents an arbitrary term. 

##### Inverse document frequency (idf)
A measure that tells us whether a term is "specific" to a document or found in a lot of documents. idf scales up rare words giving them more importance and scales down common words giving them less importance.

It is measured as,

$$
\begin{equation}
\mathrm{idf}(t, D) = \log{\frac{|D|}{n_t}}
\end{equation}
$$

where 't' is a term, 'D' is the set of all documents (corpus) and '|D|' is the total number of documents, and 'n sub t' is the number of documents where 't' appears (or tf(t, d) =/= 0)

##### tf-idf score
Measured as the product of tf and idf. Tells us how important a term is in a passage.

$$
\begin{equation}
\mathrm{tfidf}(t, d, D) = \mathrm{tf}(t, d) \cdot \mathrm{idf}(t, D)
\end{equation}
$$

##### Singular-value decomposition
A way of factorising a matrix into a product of three matrices. We use this later on to produce a low-rank approximation of a term-document matrix.

*Formula ignoring cases involving imaginary numbers*

$$
\begin{equation}
\small{
\mathbf {M} = \mathbf {U} \boldsymbol{\Sigma } \mathbf {V} ^{T}
\\
\textrm{where } \mathbf {M} \textrm{ is an m × n matrix,}
\\
\mathbf{U} \textrm{ is an m × m matrix whose columns are called the left-singular vectors of } \mathbf{M}
\\
\boldsymbol{\Sigma} \textrm{ is an m × n "diagonal" matrix whose diagonal entries are the singular values of } \mathbf{M} \textrm{ and}
\\
\mathbf {V}^{T} \textrm{ is an n × n matrix whose rows are called the right-singular vectors of } \mathbf{M}
}
\end{equation}
$$

In Python, we can easily perform SVD by using scipy's built-in `linalg.svd() method`

In [ ]:
# imports
import numpy as np
from scipy.linalg import svd 

In [ ]:
# wikipedia example
M = np.array([[1, 0, 0, 0, 2],
              [0, 0, 3, 0, 0],
              [0, 0, 0, 0, 0],
              [0, 2, 0, 0, 0]])
U, S, Vt = svd(M)

In [ ]:
U # a 4x4 matrix

In [ ]:
S # an array of the singular values in descending order

In [ ]:
Vt # a 5x5 matrix

In [ ]:
# Now we generate Sigma
m = M.shape[0] # no. of rows in M
n = M.shape[1] # no. of columns in M
Sigma = np.zeros([m, n]) # for now, an mxn zero matrix
Sigma

In [ ]:
# Populate Sigma with singular values in correct positions
for i, v in enumerate(S):
    Sigma[i, i] = v # singular value
Sigma # notice the diagonal entries

We can truncate (ignore a few dimensions in) U, Sigma, and Vt to produce an approximation of M. This is used as a form of data-reduction and solves certain issues that we come across with topic models. 

---

# What are some examples of topic models?

### Latent Semantic Analysis (LSA)

###### What is it?

A technique to figure out the relationship between a document and the terms present in the document. It is based on the assumption that similar terms appear in similar pieces of text. Given a term, LSA can be used to generate other terms that are relevant to it. 

LSA is used in search engine optimisation where you can obtain key-words that are relevant to a query. These key-words can be used by search engines to understand the content of a page and its relevance to the original query.

LSA attempts to compare the concepts represented by terms rather than the terms itself. The terms and documents are mapped into a 'concept' space where the comparisons are performed.

We do not consider the order in which the terms appear in a document (bag of words model).

We will use SVD in this method.

Overview
1. Prepare text for analysis by removing symbols, stop-words, etc.
2. Obtain document-term matrix
3. Change entries in the matrix from raw count to tf-idf scores
4. Perform Singular-Value Decomposition on this matrix to get 3 different matrices, U, Sigma, and Vt
5. Remove a few dimensions from the matrix
6. 

(ref.
http://www.scholarpedia.org/article/Latent_semantic_analysis

https://technowiki.wordpress.com/2011/08/27/latent-semantic-analysis-lsa-tutorial/)

##### How to perform LSA in Python?

Let us take a few sample articles to work with.

In [ ]:
corpus = [
    'if you\'re from Syria and you\'re a Christian, you cannot come into this country as a refugee says Trump',
    'video shows federal troops in armored vehicles patrolling the streets of Chicago on President Trump\'s orders.',
    'Donald Trump wrote in \The Art of the Deal\ that he punched out his second grade music teacher.',
    'Actor Denzel Washington said electing President Trump saved the US from becoming an \"Orwellian police state.\"',
    "President Trump fired longtime White House butler Cecil Gaines for disobedience.",
    'Congress has approved the creation of a taxpayer-funded network called \"Trump TV.\"',
    'The Islamic State "just built a hotel in Syria", according to President Donald Trump',
    "A Gucci ensemble worn by Trump counselor Kellyanne Conway to the inauguration closely resembled a 1970s 'Simplicity' pattern.,"
    'Says Melania Trump hired exorcist to \"cleanse White House of Obama demons.\"',
    '"\"Russia, Iran, Syria & many others are not happy\" about US troops leaving Syria according to the US President.'
    "In January 2019, President Donald Trump ordered FEMA to stop or cancel funding for its disaster assistance efforts in California.",
    "Trump looking to open up E Coast & new areas for offshore oil drilling when Congress has passed no new safety standards since BP",
    '"You were here long before any of us were here, although we have a representative in Congress who, they say, was here a long time ago. They call her Pocahontas.", said Trump',
    "A photograph shows an elephant carrying a lion cub.",
    "Elephant carrying thirsty lion cub",
    "Nike makes their sneakers with elephant skins.",
    "A photograph shows a jumping baby elephant.",
    "Is this a video of an elephant trampling a man to death in India?",
    "The lion used for the original MGM logo killed its trainer and his assistants.",
    "A friend and I are arguing about the origin of the photo of Donald Trump, Ivanka, and Barron with Barron sitting on the stuffed lion. She swears it's a photo-shopped, fake photo. Do you have any idea who took it or published it first?",
    "A photograph shows a real baby platypus.",
    "Photograph shows a drop bear cub being fed human blood."
]

Our first step is to convert these articles into a term-document matrix. This involves a few preparatory steps.

In [ ]:
# imports
import nltk, re
from math import log
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
# preparing data before conversion to matrix

# define stopwords
stop_words = stopwords.words('english')
stop_words += ['comes', 'says', 'according', 'come', 'shows', 'show', 'cannot']

for i in range(len(corpus)):
    
    # remove symbols and numbers
    document = re.sub('[^a-zA-Z]', ' ', corpus[i])
    
    # change to lower case
    document = document.lower()
    
    # convert string to a list of strings
    document = document.split()
    
    # remove stopwords and perform lemmatisation
    lem = WordNetLemmatizer()
    
    document = [lem.lemmatize(word) for word in document if (not word in  
            stop_words) and (len(word) > 1)]
    
    corpus[i] = document

In [ ]:
corpus

In [ ]:
# let's create a dictionary that has keys as terms and values
# as the list of documents it is found in

# the documents here can be mapped to [0, 1, 2, 3, 4, ...]

term_dict = {}
for d in range(len(corpus)):
    document = corpus[d]
    for term in document:
        if term not in term_dict:
            term_dict[term] = [d]
        else:
            term_dict[term].append(d)
            
term_dict

In [ ]:
# extract only those words that appear in more than one document.
new_term_dict = {}
for term in term_dict:
    if len(term_dict[term]) >1:
        new_term_dict[term] = term_dict[term]
new_term_dict
new_term_dict = term_dict

In [ ]:
# the length of each list within the dict gives us the frequency of the term in the corpus

# let us create a list of all the words in this dictionary that appear in two or more documents
keys = []
for term in term_dict:
    if len(term_dict[term]) >1:
        keys += [term]
keys

In [ ]:
# now we can create the term-document matrix
m = len(new_term_dict) # the number of rows in the matrix is the number of unique terms
n = len(corpus) # the number of columns is the number of documents
A = np.zeros([m, n])
A.shape

In [ ]:
# populate the matrix with the term frequencies
for index, term in enumerate(list(new_term_dict)):
    for d in new_term_dict[term]:
        A[index, d] += 1
A[:5]

In [ ]:
# now we convert term frequencies to tfidf scores
rows = A.shape[0]
cols = A.shape[1]

# array of sums of all entries in each column
col_sums = np.sum(A, axis=0)

B = np.copy(A)

for i in range(rows):

    for j in range(cols):
        
        # total number of terms in the doc j
        n = col_sums[j]
        tf = B[i, j] / n
        
        # get ith row
        row_i = list(B[i])
        
        # filter out documents that do not have the term
        row_i = [d for d in row_i if d > 0]
        
        # number of documents that have the term
        nt = len(row_i)

        
        idf = log(float(cols)) / (nt)
        
        A[i, j] = tf * idf
        
A[:5]

In [ ]:
# time to use SVD to create a low-rank approximation of our matrix
# and prepare for comparisons

# svd makes the best possible reconstruction of the matrix with the least possible information.

U, S, Vt = svd(A)

In [ ]:
# U gives us coordinates of each term in our new 'concept' space

print(U.shape)
U

In [ ]:
# S tells us how many dimensions we can include when truncating

print(S.shape)
S

In [ ]:
# Vt gives us the coordinates of each document in the 'concept' space

print(Vt.shape)
Vt

Now we need to decide which dimensions to consider.

For documents, the first dimension correlates with the length of the document. For words, it correlates with the number of times that word has been used in all documents.

So we will remove the first dimension from our matrices.

In [ ]:
# now we truncate these three matrices
k = 2 # the number of dimensions in our approximation
# the optimal number is around 250
# but we do not have that many columns

In [ ]:
Uk = U[:, 1:k+1] # all rows and 1-k+1 columns
print(Uk.shape)
Uk

In [ ]:
Sk = S[1:k+1]
print(Sk.shape)
Sk

In [ ]:
Vtk = Vt[1:k+1, :] # k rows and all columns
print(Vtk.shape)
Vtk.T

In [ ]:
# we can plot the relationship between words and documents
import matplotlib.pyplot as plt
import random

In [ ]:
%matplotlib notebook

# we have only used 2 dimensions and so we could plot this in an xy-plane

term_x = [list(Uk)[i][0] for i in range(len(list(Uk)))]
term_y = [list(Uk)[i][1] for i in range(len(list(Uk)))]

doc_x = [list(Vtk.T)[i][0] for i in range(len(list(Vtk.T)))]
doc_y = [list(Vtk.T)[i][1] for i in range(len(list(Vtk.T)))]

for index, term in enumerate(list(new_term_dict)):
    x = term_x[index]
    y = term_y[index]
    plt.scatter(x, y, color='red')
    plt.text(x+0.001, y+0.001, term, fontsize=8)
for i in range(len(corpus)):
    x = doc_x[i]
    y = doc_y[i]
    plt.scatter(x, y, color='blue')
    plt.text(x+0.0001, y+0.0001, f'doc{i}', fontsize=12)
plt.show()

Now, let's use gensim to perform LSA

In [ ]:
from gensim import corpora
from gensim.models import LsiModel

In [ ]:
# create a term dictionary of our corpus
# the keys are terms and values are integer ids
dictionary = corpora.Dictionary(corpus)
print(dictionary)

In [ ]:
# create doc-term matrix
doc_term = [dictionary.doc2bow(doc) for doc in corpus]

In [ ]:
# create instance of LSA model and pass req. params
n = 3 # number of topics
lsa = LsiModel(doc_term, num_topics=n, id2word=dictionary)

In [ ]:
lsa.print_topics()

### Latent Dirichlet Allocation

##### What is it?

(Pronounced Dirishlay or Diriklay)

ref. http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/


Most commonly used topic model.

LDA looks at a document as a mixture of topics that spit out words with certain probabilities. To generate a document, decide how many words it will have, choose a mixture of topics (the mixture can be fractional), and then generate each word by picking a topic from the mixture according to the fractional probability and use it to generate a topic-related word.

In Python, we can use the gensim library to perform LDA

Say we have a document that said "I like apples and oranges", and another that said "I like dogs and cats". We need to specify how many topics we thing are in the corpus. Here, it is evident that there are two topics atleast - 'fruits' and 'animals' say.

As we did in LSA, we create a document-term matrix. And then two other matrices, a topic-document matrix and a term-topic matrix.

We randomly assign each term in the document to one of the topics. 

For each term, we look at two things.
1. The number of times a topic appears in a document.
2. The number of times a term appears in a topic.

Say a term in our document was randomly assigned to a topic A initially, but we find that topic A does not appear a lot in the document and that the term does not appear a lot in the topic then we can change the topic of the term to something else.

We make appropiate changes to the two matrices.

After multiple iterations, we reach a point in which the topic assignment stops changing. At this point we can say that the correct topics have been assigned to each term and each document.

In [ ]:
A # this is our document-term matrix

In [ ]:
K = 3 # number of topics

In [ ]:
from gensim import matutils, models
from scipy import sparse

In [ ]:
A.T[:5] # this is our term-document matrix

In [ ]:
# convert matrix to compressed sparse matrix format
# i believe it helps in making computation easier
A_sparse = sparse.csr_matrix(A.T)
A_sparse

In [ ]:
# convert to gensim suitable format
corpus = matutils.Sparse2Corpus(A_sparse)
corpus

In [ ]:
# another input needed for the model is a dict of all terms and 
# their positions in the matrix

id2word = dict((index, list(term_dict)[index]) for index in range(len(term_dict))) # temp
id2word

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=250) # 2 topics
lda.print_topics()

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=250) # 3 topics
lda.print_topics()

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=250) # 4 topics
lda.print_topics()

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, passes=250) # 5 topics
lda.print_topics()

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=6, passes=250) # 6 topics
lda.print_topics()